#### Importing the necessary libraries

In [1]:
import requests
import os
import pandas as pd
import tweepy
import json


### Below cell contains two functions:
> - new_folder() to create new folder
> - get_files() to download files from the internet programmticly 




In [2]:

def new_folder(folder_name): 
    """
    this funcation creates a new directory in the currnet running directory when it's called
    """
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)


def get_files(file_name, url):
    """
    The following function download any file from the internet using requests library and save it in the 'dataset' directory. 
    note about the file_name: 
    the file_name var has to include the file format


    """ 
    response = requests.get(url)
    with open(os.path.join('dataset', url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)


In [3]:
new_folder('dataset')

In [39]:
URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv'

data = get_files(file_name = "twitter-archive-enhanced.csv", url= URL  )
df = pd.read_csv('dataset'+ '/'+'twitter-archive-enhanced.csv', parse_dates=True)
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

data = get_files(file_name = "image-predictions.tsv", url= URL)

In [6]:
df[df.tweet_id ==677187300187611136]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1799,677187300187611136,NaN,NaN,2015-12-16 18:03:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we see a Byzantine Rigatoni. Very aerodyn...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/677187300...,9,10,None,None,None,None,None


### Setting up tweetpy 

In [7]:

ACCESS_TOKEN = "1235133671851601920-sN4bWU6RDTydw57nqVDxwQfAvzam2J"  
ACCESS_TOKEN_SECRET = "zySb7dmbCFRKOIE1RyK07NhUC16ZGWgd9lvFJoQmtFbuL"  
CONSUMER_KEY = "JFo7yOJ5q4OssR9LNuekFJmZh" 
CONSUMER_SECRET = "5npceXl51epMWcIeqc4b20ss8FqOeCEX2NkpY1nniR7Zf69uBM"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

> The following block of code will collect three pieces of data about each tweet
> and these pieces of data are:
>    - tweet_id (will be brought from 'twitter-archive-enhanced.csv'
>    - retweet_count (will be collected using tweetpy.api)
>    - favorite_count (will be collected using tweetpy.api)

>These pieces of data will be added to a dict var in order to save them in json format


In [9]:
data = {}
data['tweet_info']=  []
print('{:20s}{:15}{}'.format("Tweet id", "retweet count",'Likes count'))
for i in df['tweet_id']:
    try:
        tweet = api.get_status(i)
        print('{:}{:10d}{:15}'.format(i, tweet.retweet_count,tweet.favorite_count))
        data['tweet_info'].append({
            'tweet_id' : i ,
            'retweet_count': tweet.retweet_count ,
            'likes_count' : tweet.favorite_count
        })
    except:
        print("Id was not found")


Tweet id            retweet count  Likes count
892420643555336193      7189          34534
892177421306343426      5397          29938
891815181378084864      3562          22533
891689557279858688      7413          37774
891327558926688256      7948          36083
891087950875897856      2672          18196
890971913173991426      1715          10601
890729181411237888     16179          58133
890609185150312448      3712          25057
890240255349198849      6248          28570
890006608113172480      6297          27594
889880896479866881      4276          25087
889665388333682689      8561          42977
889638837579907072      3818          24187
889531135344209921      1926          13630
889278841981685760      4559          22596
888917238123831296      3838          26165
888804989199671297      3630          22920
888554962724278272      2951          17696
Id was not found
888078434458587136      2971          19552
887705289381826560      4643          27193
887517139158

855851453814013952     16512          42805
855818117272018944      4934          24785
855459453768019968      7466          27695
855245323840757760      5407              0
855138241867124737        39              0
854732716440526848      5518          21192
854482394044301312      6185          27206
854365224396361728      4266          17589
854120357044912130      6791          29842
854010172552949760      2872          15177
853760880890318849      5253          26678
853639147608842240      9111          32298
853299958564483072      3246          14483
852936405516943360      1828              0
852912242202992640      1649           8438
852672615818899456      1983          13971
852553447878664193      3196          15237
852311364735569921      9104          31039
852226086759018497      6250          18735
852189679701164033      1453          10785
Id was not found
851861385021730816        19              0
851591660324737024      3211          15291
851464819735769

Rate limit reached. Sleeping for: 346


832040443403784192      9078              0
832032802820481025      4409          17154
831939777352105988      5812          23032
831926988323639298        34            324
831911600680497154      6143          26454
831670449226514432      1725          10019
831650051525054464      1828           6854
831552930092285952      2213           8649
831322785565769729      1481           8948
831315979191906304      1057           6295
831309418084069378      2331          11350
831262627380748289      1949          11440
830956169170665475      1416           7684
830583320585068544     15829          64031
830173239259324417      5378              0
830097400375152640      2775           9374
829878982036299777      4739              0
829861396166877184      1834          11801
829501995190984704     10040          30562
829449946868879360      1911          10093
Id was not found
829141528400556032      6956          23432
829011960981237760     15535          51400
828801551087042

808001312164028416      3396          12161
807621403335917568      3474          14177
807106840509214720     53079         114431
807059379405148160      7061              0
807010152071229440      3721          12763
806629075125202948     33222          72318
806620845233815552      5298              0
806576416489959424      1882           4754
806542213899489280      2275           9931
806242860592926720     11164              0
806219024703037440      1157           6278
805958939288408065      5147              0
805932879469572096      1852           8078
805826884734976000      1753           6386
805823200554876929      7703              0
805520635690676224      1596           5597
805487436403003392      2434           8557
805207613751304193      1666           7636
804738756058218496      3687          13295
804475857670639616      1945           5974
804413760345620481      3098              0
804026241225523202     15470          43108
803773340896923648      2635    

...

676946864479084545       337           1612
676942428000112642       686           1927
676936541936185344      4568          11899
676916996760600576      1671           2762
676897532954456065       667           2098
676864501615042560       650           1952
676821958043033607     14223          21276
676819651066732545       611           1696
676811746707918848       396           1341
676776431406465024      1856           4653
676617503762681856       881           2684
676613908052996102       184           1012
676606785097199616       399           1772
676603393314578432       355           1083
676593408224403456      1971           4251
676590572941893632       115            861
676588346097852417       721           2141
676582956622721024       253           1106
676575501977128964      1029           2396
676533798876651520       512           1746
676496375194980353       485           1418
676470639084101634      4306          10837
676440007570247681       610    

672622327801233409       435           1181
672614745925664768       527           1118
672609152938721280       350           1001
672604026190569472       357            998
672594978741354496       548           1204
672591762242805761       298            868
672591271085670400       170            658
672538107540070400       325            929
672523490734551040       151            563
672488522314567680       390           1019
672482722825261057       540           1025
672481316919734272       118            659
672475084225949696       633           1322
672466075045466113       473           1229
672272411274932228      3034           5880
672267570918129665       552           1354
672264251789176834       299           1032
672256522047614977      1369           2585
672254177670729728       675           1308
672248013293752320       537           1579
672245253877968896       140            622
672239279297454080       288            826
672231046314901505       915    

Rate limit reached. Sleeping for: 744


669970042633789440        49            276
669942763794931712       143            458
669926384437997569        87            346
669923323644657664        53            210
669753178989142016       354            727
669749430875258880        60            231
669684865554620416        78            458
669683899023405056        98            345
669682095984410625       123            319
669680153564442624       257            604
669661792646373376       381            730
669625907762618368      1622           3205
669603084620980224       327            870
669597912108789760       137            462
669583744538451968       834           1354
669573570759163904       125            396
669571471778410496       916           1451
669567591774625800        47            211
669564461267722241       108            353
669393256313184256        65            324
669375718304980992       649           1204
669371483794317312       155            445
669367896104181761       137    

666353288456101888        61            186
666345417576210432       127            256
666337882303524864        80            169
666293911632134144       301            440
666287406224695296        57            126
666273097616637952        69            153
666268910803644416        32             90
666104133288665088      5627          12924
666102155909144576        11             67
666099513787052032        55            137
666094000022159362        65            146
666082916733198337        39             97
666073100786774016       136            277
666071193221509120        51            130
666063827256086533       185            417
666058600524156928        49            102
666057090499244032       114            258
666055525042405380       204            386
666051853826850816       728           1053
666050758794694657        51            118
666049248165822465        37             92
666044226329800704       118            252
666033412701032449        37    

In [10]:
# Create a text file in the current directory and save each tweet's retweet count and favorite ("like") count
# in text file as json format

with open('tweet_json.txt', 'w') as outfile:
    json.dump(data, outfile)

In [48]:
# Read 'tweet.txt' line by line
with open('tweet_json.txt') as json_file:
    data = json.load(json_file)
    for p in data['tweet_info']:
        print('Tweet id: ' , p['tweet_id'])
        print('Retweet count: ' , p['retweet_count'])
        print('Likes count: ' , p['likes_count'])
        print('')

Tweet id:  892420643555336193
Retweet count:  7189
Likes count:  34534

Tweet id:  892177421306343426
Retweet count:  5397
Likes count:  29938

Tweet id:  891815181378084864
Retweet count:  3562
Likes count:  22533

Tweet id:  891689557279858688
Retweet count:  7413
Likes count:  37774

Tweet id:  891327558926688256
Retweet count:  7948
Likes count:  36083

Tweet id:  891087950875897856
Retweet count:  2672
Likes count:  18196

Tweet id:  890971913173991426
Retweet count:  1715
Likes count:  10601

Tweet id:  890729181411237888
Retweet count:  16179
Likes count:  58133

Tweet id:  890609185150312448
Retweet count:  3712
Likes count:  25057

Tweet id:  890240255349198849
Retweet count:  6248
Likes count:  28570

Tweet id:  890006608113172480
Retweet count:  6297
Likes count:  27594

Tweet id:  889880896479866881
Retweet count:  4276
Likes count:  25087

Tweet id:  889665388333682689
Retweet count:  8561
Likes count:  42977

Tweet id:  889638837579907072
Retweet count:  3818
Likes count:

Likes count:  14191

Tweet id:  836260088725786625
Retweet count:  4089
Likes count:  20427

Tweet id:  836001077879255040
Retweet count:  4013
Likes count:  18309

Tweet id:  835685285446955009
Retweet count:  7661
Likes count:  0

Tweet id:  835574547218894849
Retweet count:  3394
Likes count:  17052

Tweet id:  835536468978302976
Retweet count:  1585
Likes count:  0

Tweet id:  835309094223372289
Retweet count:  19716
Likes count:  0

Tweet id:  835297930240217089
Retweet count:  2823
Likes count:  15805

Tweet id:  835264098648616962
Retweet count:  1585
Likes count:  7403

Tweet id:  835246439529840640
Retweet count:  66
Likes count:  2020

Tweet id:  835172783151792128
Retweet count:  5416
Likes count:  25122

Tweet id:  835152434251116546
Retweet count:  2832
Likes count:  21456

Tweet id:  834931633769889797
Retweet count:  1521
Likes count:  10338

Tweet id:  834786237630337024
Retweet count:  5216
Likes count:  20639

Tweet id:  834574053763584002
Retweet count:  2368
Likes c

Retweet count:  6404
Likes count:  0

Tweet id:  798701998996647937
Retweet count:  7453
Likes count:  0

Tweet id:  798697898615730177
Retweet count:  6267
Likes count:  0

Tweet id:  798694562394996736
Retweet count:  4772
Likes count:  0

Tweet id:  798686750113755136
Retweet count:  2241
Likes count:  0

Tweet id:  798682547630837760
Retweet count:  4569
Likes count:  0

Tweet id:  798673117451325440
Retweet count:  5365
Likes count:  0

Tweet id:  798665375516884993
Retweet count:  3770
Likes count:  0

Tweet id:  798644042770751489
Retweet count:  1785
Likes count:  0

Tweet id:  798628517273620480
Retweet count:  1903
Likes count:  0

Tweet id:  798585098161549313
Retweet count:  5445
Likes count:  0

Tweet id:  798576900688019456
Retweet count:  5627
Likes count:  0

Tweet id:  798340744599797760
Retweet count:  3219
Likes count:  0

Tweet id:  798209839306514432
Retweet count:  2472
Likes count:  10151

Tweet id:  797971864723324932
Retweet count:  2989
Likes count:  11251

Tw

Likes count:  10128

Tweet id:  771171053431250945
Retweet count:  7069
Likes count:  0

Tweet id:  771136648247640064
Retweet count:  2779
Likes count:  8802

Tweet id:  771102124360998913
Retweet count:  1369
Likes count:  5994

Tweet id:  771014301343748096
Retweet count:  1576
Likes count:  6638

Tweet id:  770787852854652928
Retweet count:  1144
Likes count:  4747

Tweet id:  770772759874076672
Retweet count:  1354
Likes count:  4987

Tweet id:  770655142660169732
Retweet count:  1706
Likes count:  7087

Tweet id:  770414278348247044
Retweet count:  1989
Likes count:  6093

Tweet id:  770293558247038976
Retweet count:  1391
Likes count:  5972

Tweet id:  770093767776997377
Retweet count:  2894
Likes count:  0

Tweet id:  770069151037685760
Retweet count:  2173
Likes count:  7304

Tweet id:  769940425801170949
Retweet count:  9078
Likes count:  30033

Tweet id:  769695466921623552
Retweet count:  1599
Likes count:  6182

Tweet id:  769335591808995329
Retweet count:  7336
Likes coun

Retweet count:  4679
Likes count:  12327

Tweet id:  735256018284875776
Retweet count:  825
Likes count:  3146

Tweet id:  735137028879360001
Retweet count:  908
Likes count:  2954

Tweet id:  734912297295085568
Retweet count:  493
Likes count:  2607

Tweet id:  734787690684657664
Retweet count:  5921
Likes count:  11853

Tweet id:  734776360183431168
Retweet count:  501
Likes count:  2378

Tweet id:  734559631394082816
Retweet count:  370
Likes count:  1407

Tweet id:  733828123016450049
Retweet count:  734
Likes count:  3377

Tweet id:  733822306246479872
Retweet count:  962
Likes count:  3484

Tweet id:  733482008106668032
Retweet count:  883
Likes count:  2947

Tweet id:  733460102733135873
Retweet count:  1175
Likes count:  3958

Tweet id:  733109485275860992
Retweet count:  15960
Likes count:  50537

Tweet id:  732732193018155009
Retweet count:  499
Likes count:  2294

Tweet id:  732726085725589504
Retweet count:  827
Likes count:  3361

Tweet id:  732585889486888962
Retweet coun


Tweet id:  700462010979500032
Retweet count:  1669
Likes count:  3884

Tweet id:  700167517596164096
Retweet count:  683
Likes count:  2495

Tweet id:  700151421916807169
Retweet count:  631
Likes count:  2114

Tweet id:  700143752053182464
Retweet count:  2530
Likes count:  7118

Tweet id:  700062718104104960
Retweet count:  639
Likes count:  2503

Tweet id:  700029284593901568
Retweet count:  544
Likes count:  1968

Tweet id:  700002074055016451
Retweet count:  1254
Likes count:  3128

Tweet id:  699801817392291840
Retweet count:  881
Likes count:  2858

Tweet id:  699788877217865730
Retweet count:  483
Likes count:  2118

Tweet id:  699779630832685056
Retweet count:  1156
Likes count:  2585

Tweet id:  699775878809702401
Retweet count:  562
Likes count:  1856

Tweet id:  699691744225525762
Retweet count:  4299
Likes count:  9719

Tweet id:  699446877801091073
Retweet count:  2460
Likes count:  5655

Tweet id:  699434518667751424
Retweet count:  480
Likes count:  2073

Tweet id:  69

Likes count:  2894

Tweet id:  682259524040966145
Retweet count:  1110
Likes count:  3907

Tweet id:  682242692827447297
Retweet count:  1074
Likes count:  3047

Tweet id:  682088079302213632
Retweet count:  9169
Likes count:  17217

Tweet id:  682059653698686977
Retweet count:  1574
Likes count:  4657

Tweet id:  682047327939461121
Retweet count:  875
Likes count:  3025

Tweet id:  682032003584274432
Retweet count:  1863
Likes count:  6217

Tweet id:  682003177596559360
Retweet count:  1411
Likes count:  2942

Tweet id:  681981167097122816
Retweet count:  954
Likes count:  2649

Tweet id:  681891461017812993
Retweet count:  767
Likes count:  2332

Tweet id:  681694085539872773
Retweet count:  3798
Likes count:  12118

Tweet id:  681679526984871937
Retweet count:  414
Likes count:  1675

Tweet id:  681654059175129088
Retweet count:  844
Likes count:  2477

Tweet id:  681610798867845120
Retweet count:  435
Likes count:  1784

Tweet id:  681579835668455424
Retweet count:  1211
Likes coun

Tweet id:  673363615379013632
Retweet count:  273
Likes count:  949

Tweet id:  673359818736984064
Retweet count:  612
Likes count:  1324

Tweet id:  673355879178194945
Retweet count:  511
Likes count:  1375

Tweet id:  673352124999274496
Retweet count:  485
Likes count:  1531

Tweet id:  673350198937153538
Retweet count:  190
Likes count:  698

Tweet id:  673345638550134785
Retweet count:  1141
Likes count:  2286

Tweet id:  673343217010679808
Retweet count:  245
Likes count:  884

Tweet id:  673342308415348736
Retweet count:  512
Likes count:  1167

Tweet id:  673320132811366400
Retweet count:  7069
Likes count:  12283

Tweet id:  673317986296586240
Retweet count:  225
Likes count:  805

Tweet id:  673295268553605120
Retweet count:  2841
Likes count:  6900

Tweet id:  673270968295534593
Retweet count:  326
Likes count:  979

Tweet id:  673240798075449344
Retweet count:  636
Likes count:  1305

Tweet id:  673213039743795200
Retweet count:  754
Likes count:  2033

Tweet id:  6731488042


Tweet id:  667878741721415682
Retweet count:  106
Likes count:  358

Tweet id:  667873844930215936
Retweet count:  364
Likes count:  567

Tweet id:  667866724293877760
Retweet count:  926
Likes count:  2771

Tweet id:  667861340749471744
Retweet count:  73
Likes count:  225

Tweet id:  667832474953625600
Retweet count:  56
Likes count:  263

Tweet id:  667806454573760512
Retweet count:  441
Likes count:  956

Tweet id:  667801013445750784
Retweet count:  83
Likes count:  292

Tweet id:  667793409583771648
Retweet count:  301
Likes count:  631

Tweet id:  667782464991965184
Retweet count:  220
Likes count:  372

Tweet id:  667773195014021121
Retweet count:  46
Likes count:  212

Tweet id:  667766675769573376
Retweet count:  195
Likes count:  388

Tweet id:  667728196545200128
Retweet count:  134
Likes count:  334

Tweet id:  667724302356258817
Retweet count:  283
Likes count:  436

Tweet id:  667550904950915073
Retweet count:  29
Likes count:  0

Tweet id:  667550882905632768
Retweet c

In [49]:
# Read 'tweet.txt' line by line into a pandas dataframe with tweet_id, retweet_count, and likes_count
tweet_json_df = pd.json_normalize(data['tweet_info'])


In [47]:
tweet_json_df.head()

,tweet_id,retweet_count,likes_count
0,892420643555336193,7189,34534
1,892177421306343426,5397,29938
2,891815181378084864,3562,22533
3,891689557279858688,7413,37774
4,891327558926688256,7948,36083
